### RAG chatbot with ChromaDB

In [6]:
import os
from dotenv import load_dotenv
load_dotenv()

True

In [7]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import TextLoader
from langchain_community.document_loaders import PyPDFLoader

from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_community.embeddings import HuggingFaceBgeEmbeddings

from langchain_community.vectorstores import Chroma
import numpy as np
from typing import List
import os
from dotenv import load_dotenv

In [8]:
pdf_path = "latest_tech_news.pdf"

loader = PyPDFLoader(pdf_path)
documents = loader.load()

print(f"Documents Loaded!!! {len(documents)} pages!")
documents

Documents Loaded!!! 2 pages!


[Document(metadata={'producer': 'ReportLab PDF Library - www.reportlab.com', 'creator': '(unspecified)', 'creationdate': '2025-09-25T09:24:19+00:00', 'author': '(anonymous)', 'keywords': '', 'moddate': '2025-09-25T09:24:19+00:00', 'subject': '(unspecified)', 'title': '(anonymous)', 'trapped': '/False', 'source': 'latest_tech_news.pdf', 'total_pages': 2, 'page': 0, 'page_label': '1'}, page_content="Latest AI & Tech Industry News — Global & India (as of\n September 25, 2025)\nDate: September 25, 2025\nOverview\nThis briefing summarizes major, recent developments in artificial intelligence (AI) and the global\ntechnology industry, plus focused coverage of important technology news from India. It pulls together\nreporting and analysis from leading outlets and policy organizations to give you a compact, actionable\npicture of where the sector stands today.\n1. Major AI industry developments (global)\n\x7f Microsoft expanded the model choices inside Microsoft 365 Copilot by integrating Anthr

In [9]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=100,
    length_function=len,
    separators=["\n\n", "\n", ". ", ", ", " ", ""] # Heirarchy of splitters
)

docs_chunks = text_splitter.split_documents(documents)

print(f"Created chunks ===== {len(docs_chunks)}")
print(f"Doc 0's content ===== {docs_chunks[0].page_content[:]}")
docs_chunks

Created chunks ===== 10
Doc 0's content ===== Latest AI & Tech Industry News — Global & India (as of
 September 25, 2025)
Date: September 25, 2025
Overview
This briefing summarizes major, recent developments in artificial intelligence (AI) and the global
technology industry, plus focused coverage of important technology news from India. It pulls together
reporting and analysis from leading outlets and policy organizations to give you a compact, actionable
picture of where the sector stands today.
1. Major AI industry developments (global)


[Document(metadata={'producer': 'ReportLab PDF Library - www.reportlab.com', 'creator': '(unspecified)', 'creationdate': '2025-09-25T09:24:19+00:00', 'author': '(anonymous)', 'keywords': '', 'moddate': '2025-09-25T09:24:19+00:00', 'subject': '(unspecified)', 'title': '(anonymous)', 'trapped': '/False', 'source': 'latest_tech_news.pdf', 'total_pages': 2, 'page': 0, 'page_label': '1'}, page_content='Latest AI & Tech Industry News — Global & India (as of\n September 25, 2025)\nDate: September 25, 2025\nOverview\nThis briefing summarizes major, recent developments in artificial intelligence (AI) and the global\ntechnology industry, plus focused coverage of important technology news from India. It pulls together\nreporting and analysis from leading outlets and policy organizations to give you a compact, actionable\npicture of where the sector stands today.\n1. Major AI industry developments (global)'),
 Document(metadata={'producer': 'ReportLab PDF Library - www.reportlab.com', 'creator': '

In [ ]:
embedding_model = HuggingFaceBgeEmbeddings(
        model_name="sentence-transformers/all-MiniLM-L6-v2"
    )

persist_directory="./chroma_db"

vector_store = Chroma.from_documents(
    documents=docs_chunks,
    embedding=embedding_model,
    persist_directory=persist_directory,
    collection_name="rag_chroma_collection"
)

print(f"Vector store created with {vector_store._collection.count()} vectors")
print(f"Vector store directory === {persist_directory}")
  

C:\Users\gaurav\AppData\Local\Temp\ipykernel_18812\4142187143.py:1: LangChainDeprecationWarning: The class `HuggingFaceBgeEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_model = HuggingFaceBgeEmbeddings(
c:\Practice\RAG\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Vector store created with 50 vectors
Vector store directory === ./chroma_db


### TESTING SIMILARITY SEARCH

In [11]:
user_query = "Major AI developments and policies and regulatory moves"

similar_vectors = vector_store.similarity_search(user_query, k=3)


for sv in similar_vectors:
    print("--------------------------")
    print(sv.page_content)
 
similar_vectors


--------------------------
and investing in GPU supply chains and AI chips. This includes activity from companies building or
announcing new AI chip clusters and cloud deals.
3. Policy, regulation, and ethical considerations
 Governments and international organizations are increasingly active on AI governance: calls for binding
international action on dangerous uses of AI, national efforts for data sovereignty, and regulatory moves at
--------------------------
and investing in GPU supply chains and AI chips. This includes activity from companies building or
announcing new AI chip clusters and cloud deals.
3. Policy, regulation, and ethical considerations
 Governments and international organizations are increasingly active on AI governance: calls for binding
international action on dangerous uses of AI, national efforts for data sovereignty, and regulatory moves at
--------------------------
and investing in GPU supply chains and AI chips. This includes activity from companies buildi

[Document(metadata={'total_pages': 2, 'page_label': '1', 'source': 'latest_tech_news.pdf', 'trapped': '/False', 'title': '(anonymous)', 'author': '(anonymous)', 'keywords': '', 'moddate': '2025-09-25T09:24:19+00:00', 'creationdate': '2025-09-25T09:24:19+00:00', 'creator': '(unspecified)', 'subject': '(unspecified)', 'page': 0, 'producer': 'ReportLab PDF Library - www.reportlab.com'}, page_content='and investing in GPU supply chains and AI chips. This includes activity from companies building or\nannouncing new AI chip clusters and cloud deals.\n3. Policy, regulation, and ethical considerations\n\x7f Governments and international organizations are increasingly active on AI governance: calls for binding\ninternational action on dangerous uses of AI, national efforts for data sovereignty, and regulatory moves at'),
 Document(metadata={'creationdate': '2025-09-25T09:24:19+00:00', 'subject': '(unspecified)', 'author': '(anonymous)', 'title': '(anonymous)', 'creator': '(unspecified)', 'page'

In [12]:
user_query = "Major AI developments and policies and regulatory moves"

similar_vectors = vector_store.similarity_search_with_relevance_scores(user_query, k=3)
similar_vectors

[(Document(metadata={'author': '(anonymous)', 'subject': '(unspecified)', 'title': '(anonymous)', 'total_pages': 2, 'source': 'latest_tech_news.pdf', 'producer': 'ReportLab PDF Library - www.reportlab.com', 'page_label': '1', 'creator': '(unspecified)', 'moddate': '2025-09-25T09:24:19+00:00', 'trapped': '/False', 'page': 0, 'creationdate': '2025-09-25T09:24:19+00:00', 'keywords': ''}, page_content='and investing in GPU supply chains and AI chips. This includes activity from companies building or\nannouncing new AI chip clusters and cloud deals.\n3. Policy, regulation, and ethical considerations\n\x7f Governments and international organizations are increasingly active on AI governance: calls for binding\ninternational action on dangerous uses of AI, national efforts for data sovereignty, and regulatory moves at'),
  0.6018039931144111),
 (Document(metadata={'trapped': '/False', 'keywords': '', 'producer': 'ReportLab PDF Library - www.reportlab.com', 'source': 'latest_tech_news.pdf', 'cr

ChromaDB by default uses L2 Distance (Euclidean distance)

- Lower scores = More similar
- Score of 0 = Identical
- Usual Range = 0 to 2 (can be higher as well)


Cosine similarity 
- Higher scores = More similar
- Range = -1 to 1 (1 is identical)

In [13]:
llm = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash",
    google_api_key=os.getenv("GOOGLE_API_KEY")
)
print(llm.invoke("Capital of India is?"))

content='The capital of India is **New Delhi**.' additional_kwargs={} response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash', 'safety_ratings': []} id='run--8f63908a-5721-47ef-91d1-74b4ca93a753-0' usage_metadata={'input_tokens': 6, 'output_tokens': 26, 'total_tokens': 32, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 17}}


### Modern RAG Chain

In [14]:
from langchain.chains import create_retrieval_chain
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains.combine_documents import create_stuff_documents_chain

In order to use vector store in RAG reteival chain, we need to convert vector store into a retriever

In [15]:
retriever=vector_store.as_retriever(
    search_kwargs={"k":3}  ## Retrieve top 3 relevant chunks 
)
retriever

VectorStoreRetriever(tags=['Chroma', 'HuggingFaceBgeEmbeddings'], vectorstore=<langchain_community.vectorstores.chroma.Chroma object at 0x000001E7BC529910>, search_kwargs={'k': 3})

In [16]:
### Creating prompt template
from langchain_core.prompts import ChatPromptTemplate
system_prompt="""
You are an assistant for question-answering tasks. 
Use the following information as context to answer the questions. 
If you don't know the answer, do not hallicunate instead just say you don't know. 
Use three sentences maximum to answer the question and keep answers crisp and concise.

Context: {context}
"""

prompt = ChatPromptTemplate.from_messages([
    ("system", system_prompt),
    ("human", "{input}")
])

In [17]:
prompt

ChatPromptTemplate(input_variables=['context', 'input'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, template="\nYou are an assistant for question-answering tasks. \nUse the following information as context to answer the questions. \nIf you don't know the answer, do not hallicunate instead just say you don't know. \nUse three sentences maximum to answer the question and keep answers crisp and concise.\n\nContext: {context}\n"), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='{input}'), additional_kwargs={})])

In [18]:
### DOCUMENT CHAIN
from langchain.chains.combine_documents import create_stuff_documents_chain
document_chain=create_stuff_documents_chain(llm, prompt)
document_chain

RunnableBinding(bound=RunnableBinding(bound=RunnableAssign(mapper={
  context: RunnableLambda(format_docs)
}), kwargs={}, config={'run_name': 'format_inputs'}, config_factories=[])
| ChatPromptTemplate(input_variables=['context', 'input'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, template="\nYou are an assistant for question-answering tasks. \nUse the following information as context to answer the questions. \nIf you don't know the answer, do not hallicunate instead just say you don't know. \nUse three sentences maximum to answer the question and keep answers crisp and concise.\n\nContext: {context}\n"), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='{input}'), additional_kwargs={})])
| ChatGoogleGenerativeAI(model='models/gemini-2.5-flash', google_api_key=SecretStr

In [19]:
rag_chain=create_retrieval_chain(retriever, document_chain)
rag_chain

RunnableBinding(bound=RunnableAssign(mapper={
  context: RunnableBinding(bound=RunnableLambda(lambda x: x['input'])
           | VectorStoreRetriever(tags=['Chroma', 'HuggingFaceBgeEmbeddings'], vectorstore=<langchain_community.vectorstores.chroma.Chroma object at 0x000001E7BC529910>, search_kwargs={'k': 3}), kwargs={}, config={'run_name': 'retrieve_documents'}, config_factories=[])
})
| RunnableAssign(mapper={
    answer: RunnableBinding(bound=RunnableBinding(bound=RunnableAssign(mapper={
              context: RunnableLambda(format_docs)
            }), kwargs={}, config={'run_name': 'format_inputs'}, config_factories=[])
            | ChatPromptTemplate(input_variables=['context', 'input'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, template="\nYou are an assistant for question-answering tasks. \nUse the following information as context to answer the questions. \

In [24]:
response = rag_chain.invoke({"input":"Latest trending tech news related to AI"})
response

{'input': 'Latest trending tech news related to AI',
 'context': [Document(metadata={'producer': 'ReportLab PDF Library - www.reportlab.com', 'moddate': '2025-09-25T09:24:19+00:00', 'subject': '(unspecified)', 'page_label': '1', 'trapped': '/False', 'creationdate': '2025-09-25T09:24:19+00:00', 'title': '(anonymous)', 'page': 0, 'keywords': '', 'source': 'latest_tech_news.pdf', 'total_pages': 2, 'author': '(anonymous)', 'creator': '(unspecified)'}, page_content='Latest AI & Tech Industry News — Global & India (as of\n September 25, 2025)\nDate: September 25, 2025\nOverview\nThis briefing summarizes major, recent developments in artificial intelligence (AI) and the global\ntechnology industry, plus focused coverage of important technology news from India. It pulls together\nreporting and analysis from leading outlets and policy organizations to give you a compact, actionable\npicture of where the sector stands today.\n1. Major AI industry developments (global)'),
  Document(metadata={'cr

In [23]:
response['answer']

'This briefing summarizes major, recent developments in artificial intelligence (AI) and the global technology industry. It also includes focused coverage of important technology news from India. The information is current as of September 25, 2025.'